In [0]:
from textblob import TextBlob
import tweepy
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
consumerKey="SUyt2H9XLhjd2UkwkbkuMbdpn"
consumerSecret="LHo1I5OqPgOaxsrIWOQJW2nt2IUkbbMMPSZltEWpqE2w915LJS"
accessToken="1147374963642400770-2tU83HNNTlSzt7KLYORxcHVFcT0EZN"
accessSecret="VIoONJ9JwS1xcL95eEowqyiF0CVQQG9zMfJnEsACyrqsQ"

In [0]:
auth=tweepy.OAuthHandler(consumerKey,consumerSecret)
auth.set_access_token(accessToken,accessSecret)
api=tweepy.API(auth)

In [0]:
df = pd.DataFrame(columns = ['Tweets', 'User'])

In [0]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search,count=2000 ,q=data,lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.to_excel('{}.xlsx'.format(file_name))
        i+=1
        if i == 2000:
            break
        else:
            pass

In [0]:
stream(data = ['world'], file_name = 'my_tweet')

In [0]:
df.head()

In [0]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|RT', ' ', tweet).split())

In [0]:
import re
df['clean_tweet'] = df['Tweets'].apply(lambda x: clean_tweet(x))

In [0]:
df.head()

,Tweets,User,clean_tweet
0,"RT @TrueEyeTheSpy: Ah, it seems Patriots the w...",Leslie,Ah it seems Patriots the world over maxed out ...
1,RT @spicebae_: this the greatest feeling in th...,Queen k ♕,this the greatest feeling in the world to look...
2,RT @1984to1776: Water world Tsunami injuring m...,kurapika vs uvo 😭,Water world Tsunami injuring many in Yanbian M...
3,RT @RamenhaeMXMB: elhae's IG story after leavi...,Qamara,elhae s IG story after leaving today s monsta ...
4,RT @Joe_Thomson_: @ladbible Volkswagen launche...,저세티❄️ เรียกเราว่าจ๋าจ้า,Thomson Volkswagen launches new family car tha...


In [0]:
li=df["clean_tweet"].values.tolist()
li1=df["User"].values.tolist()

In [0]:
lis=[]
import spacy 
nlp = spacy.load('en_core_web_sm')
for i in  li:
  doc = nlp(i)
  d=[]
  for j in doc.ents:
   
    d.append(j.label_)
  if "GPE" in d:
    lis.append("1")
  else:
    lis.append("0")
    
    

In [0]:
len(lis)

2000

In [0]:
df['Value_loc']=lis

In [0]:
df.head(20)

,Tweets,User,clean_tweet,Value_loc
0,RT @bts_love_myself: Just as you love yourself...,𝙺𝚒𝚖 | 𝓵𝓸𝓿𝓮𝓼 𝓳𝓴 ㅇㅅㅇ,love myself Just as you love yourself do you l...,0
1,Good morning World ....... https://t.co/4lLOD9...,Lexi,Good morning World,0
2,"RT @chartdata: .@Beyonce, @SAINtJHN, @wizkiday...",Uncle Damola 🇳🇬🕷,and Blue Ivy Carter s BROWN SKIN GIRL debuts a...,0
3,@GulBukhari @BushraGohar We are one of the mos...,Awais Maqbool Wattoo,We are one of the most corrupt nations of the ...,0
4,"RT @teasofjenlisa: ""So stay, I know you gotta ...",yowyow,So stay I know you gotta be tired When the nig...,0
5,"RT @womensart1: 'All’s Right with the World', ...",Arnaav Bhavanani 🦔,All s Right with the World linoblock woodblock...,0
6,“Foreign language” - this concept is not fit f...,Carsten Levisen,Foreign language this concept is not fit for t...,0
7,RT @ukpcn: Photography as Record\nFREE: Talk a...,Charlie Turpie,Photography as Record FREE Talk and Document V...,0
8,RT @finalfantasyvii: Take a closer look at the...,Naedren Olórin,Take a closer look at the entrance to Shinra B...,0
9,WAS ABT TO WATCH TORADORA AGAIN BUT IT REMINDS...,janne raquel 🌸,WAS ABT TO WATCH TORADORA AGAIN BUT IT REMINDS...,0


In [0]:
X=df[['clean_tweet']]
y=df['Value_loc']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vect = CountVectorizer(binary=True)
X = vect.fit_transform(df['clean_tweet'])

X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
from sklearn.model_selection import train_test_split

X = df.clean_tweet
y = df.Value_loc

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(max_features=1000, binary=True)

X_train_vect = vect.fit_transform(X_train)

In [0]:
counts = df.Value_loc.value_counts()
print(counts)

0    1604
1     396
Name: Value_loc, dtype: int64


In [0]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(X_train_vect, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
X_test_vect = vect.transform(X_test)

In [0]:
y_pred = nb.predict(X_test_vect)

y_pred

array(['1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0',
       '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0

In [0]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report 
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))


Accuracy: 90.00%


In [0]:
cm=confusion_matrix(y_test,y_pred)
print(cm)

print(classification_report(y_test,y_pred))

[[307   9]
 [ 31  53]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       316
           1       0.85      0.63      0.73        84

    accuracy                           0.90       400
   macro avg       0.88      0.80      0.83       400
weighted avg       0.90      0.90      0.89       400

